In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import numpy as np
import pandas as pd
import time
import json
from bs4 import BeautifulSoup as bs

In [6]:

driver = webdriver.Firefox(executable_path='C:/Users/micah/geckodriver-v0.31.0-win64/geckodriver.exe')
def login(driver):
    with open("smith.txt", 'r') as f:
        account = (f.read()).split(",")
    driver.get("https://www.facebook.com/")
    elem = driver.find_element(By.XPATH, '//*[@id="email"]')
    elem.clear()
    elem.send_keys(account[0])
    passw = driver.find_element(By.XPATH,'//*[@id="pass"]')
    passw.send_keys(account[1])
    login = driver.find_element(By.XPATH,'/html/body/div[1]/div[1]/div[1]/div/div/div/div[2]/div/div[1]/form/div[2]/button')
    login.click()
def generalNameList():
    x = pd.read_csv('names.csv')
    return x['Generals'].to_list()
def nameToLink(name):
    return name.replace(" ", "%20")
def generalSearch(driver):
    generals = generalNameList()
    master_list = ""
    count = 0
    for general in generals:
        # elem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[2]/div[2]/div[1]/div/div/div[3]/div/div/div/div[1]/div/div/label/input')
        # elem.send_keys(general)
        driver.get("https://www.facebook.com/search/people/?q=" + nameToLink(general))
        textual_healing = driver.page_source
        expression = '"profile":{"__typename":"User","__isNode":"User","id":"[0-9]*"'
        ids = re.findall(expression, textual_healing)
        x = np.array(ids)
        p = np.unique(x)
        gen = (general + "/")*len(p)
        p = " ".join(p)
        gen2 = gen + ";" + p
        master_list += str(gen2) + "\n"
        if count%15 == 0 and count != 0:
            time.sleep(10)
        count +=1
    driver.close()
    with open('general_ids.txt', 'w') as f:
        f.write(str(master_list))

# login(driver)
# generalSearch(driver)

C:\Users\micah\AppData\Local\Temp\ipykernel_24324\2851653758.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/micah/geckodriver-v0.31.0-win64/geckodriver.exe')


In [3]:
def getDataFrame():
    q = open('general_ids.txt', 'r')
    text = q.read()
    text = text.split("\n")
    id_list= []
    for tex in text:
        general_list = tex.split(";")
        general_list[0] = general_list[0].split("/")
        if len(general_list) == 2:
            general_list[1] = general_list[1].split(" ")
            for i in range(len(general_list[1])):
                general_list[1][i] = re.sub('\D', '', general_list[1][i])
            q = list(zip(general_list[0], general_list[1]))
        id_list.append(q)
    actual = []
    for item in id_list:
        actual += item
    hier_index = pd.MultiIndex.from_tuples(actual)
    df = pd.DataFrame(np.zeros(len(actual)), index=hier_index, columns=['score'])
    df.index.names = ['general', 'id']
    return df

df = getDataFrame()
df

score
general          id                    
Mark W. Gillette 100021231701450    0.0
                 100030520254298    0.0
                 100045593106158    0.0
                 100054796545986    0.0
                 100057821166064    0.0
...                                 ...
Tony L. Wright   100006098828370    0.0
                 100009837580460    0.0
                 100011042771830    0.0
                 100083776454937    0.0
                 1676286596         0.0

[658 rows x 1 columns]

In [46]:
# driver = webdriver.Firefox(executable_path='C:/Users/micah/geckodriver-v0.31.0-win64/geckodriver.exe')
# login(driver)
# driver.get("https://www.facebook.com/jj.j.heck")
# source = driver.page_source
with open('text.html', 'w', encoding='utf-8') as f:
    f.write(source)
def findFriends(html):
    rex = r'"text":".*\sfriends"'
    friends = re.findall(rex,html)
    if friends == []:
        return 0
    hell = re.findall(r'(\d+(?:\.\d+)?)', friends[0])
    if 'K' in friends[0]:
        return float(hell[0])*1000
    else:
        return float(hell[0])   
with open('text.html', 'r', encoding='utf-8') as f:
    source = f.read()
def findName(html):
    soup =bs(html, 'html.parser')
    name1=soup.title.get_text()
    name2 = name1.replace('(2)', "")
    name3 = name2.replace(' | Facebook', '')
    return name3
def findInfo(html):
    soup = bs(html, 'html.parser')
    test = soup.find_all('div', class_='rq0escxv l9j0dhe7 du4w35lb j83agx80 cbu4d94t d2edcug0 hpfvmrgz rj1gh0hx buofh1pr g5gj957u o8rfisnq p8fzw8mz pcp91wgn iuny7tx3 ipjc6fyt')
    array = []
    for thing in test:
        if "instagram" not in str(thing) and'twitter' not in str(thing) and 'facebook' not in thing.get_text():
            array.append(thing.get_text())
    array = np.array(array)
    array1 = np.unique(array)
    dict = {"Hometown":"", 'Current_town':"", "Job": []}
    if array1.size == 0:
        return dict
    for item in array1:
        if 'From' in item:
            dict["Hometown"] = item.split("From")[1]
        elif " in " in item and not "Studied" in item:
            dict["Current_town"] = item.split(' in ')[1]
        elif 'Works at ' in item or 'Worked at' in item:
            dict["Job"].append(item.split(' at ')[1])
        elif 'Studied ' not in item and 'Went' not in item and 'From' not in item and 'Lives' not in item and not 'https' in item and ' in ' not in item and "Pronounces" not in item:
            if "It's" not in item and "Widowed" not in item and "Joined" not in item and 'facebook' not in item and 'Divorced' not in item:
                if ' with ' not in item and 'In ' not in item and 'relationship' not in item and 'Married' not in item and "Followed" not in item and "Engaged" not in item and "Single" not in item:
                    dict["Job"].append(item)
    return dict


In [47]:
driver = webdriver.Firefox(executable_path='C:/Users/micah/geckodriver-v0.31.0-win64/geckodriver.exe')
login(driver)
count = 0
for idx, data in df.groupby(level='id'):
    driver.get('https://www.facebook.com/profile.php?id=' + idx)
    html = driver.page_source
    friends = findFriends(html)
    info = findInfo(html)
    name = findName(html)
    print(name, info, friends)
    if count%15 == 0 and count !=0:
        time.sleep(20)
    count +=1

driver.close()


C:\Users\micah\AppData\Local\Temp\ipykernel_24324\2415142928.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/micah/geckodriver-v0.31.0-win64/geckodriver.exe')


Johnny Carr IV {'Hometown': ' East Bridgewater, Massachusetts', 'Current_town': '', 'Job': []} 0
Richard Heitkamp {'Hometown': ' Klamath Falls, Oregon', 'Current_town': 'Klamath Falls, Oregon', 'Job': ['Taco Bell']} 119.0
William Graham Jr. {'Hometown': '', 'Current_town': '', 'Job': []} 0
Todd Wasmund {'Hometown': '', 'Current_town': '', 'Job': []} 0
Neil Hersey {'Hometown': '', 'Current_town': '', 'Job': ["Bartender at Dave & Buster's"]} 507.0
Robert Marsh Jr {'Hometown': '', 'Current_town': '', 'Job': []} 476.0
 Stephen Strand {'Hometown': '', 'Current_town': '', 'Job': []} 0
Deborah Kolich Pelletier {'Hometown': ' Wheeling, West Virginia', 'Current_town': '', 'Job': []} 0
Robert Harter {'Hometown': ' Berwick, Pennsylvania', 'Current_town': 'Berwick, Pennsylvania', 'Job': []} 450.0
Patrick Robertson {'Hometown': '', 'Current_town': '', 'Job': []} 763.0
Michel Russell Sr. {'Hometown': '', 'Current_town': '', 'Job': []} 0
Robert Harper {'Hometown': '', 'Current_town': '', 'Job': []} 9

KeyboardInterrupt: 